[![](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/itmorn/AI.handbook/blob/main/DL/torch/nn/Normalization/BatchNorm.ipynb)

# 简介批量归一化
归一化的思想比较早，而在卷积网络中提出BatchNorm层是发生在2016年。

在训练阶段，我们会对每一个批次，在通道维度上求均值EX和样本方差VarX，然后对该批次数据在通道维度上进行规范化；另外BN层还有两个可以学习的参数（ $γ$ 和 $β$ ），它们可以对数据的分布进行二次矫正。再另外，训练阶段会使用移动平均的方式近似维护训练数据整体的均值（running_mean）和整体的无偏方差（running_var），以供推理时对单个样本进行分布矫正。

在推理阶段，直接使用整体的均值（running_mean）和整体的无偏方差（running_var）对数据进行规范化，再使用 $γ$ 和 $β$ 进行二次矫正。


批量归一化可以加速收敛速度，但一般不改变模型精度。

# 为什么要用BatchNorm
当神经网络深度比较深的时候，会出现梯度消失的问题，即靠近loss的层参数更新的快，靠近input的层参数更新的慢，这主要是由误差反向传播时，数据的分布不规范，可能很多数据都落在激活函数导函数比较小的位置，最终累乘的结果比较小导致的。而BN层就可以对每一层的数据分布进行矫正，从而达到稳定训练，快速收敛的效果。

# 为什么BatchNorm不与dropout同时使用
由于训练时，每一个小批量都要计算均值和方差，然后对每个样本进行规范化。这就相当于再给数据加噪声，已经可以起到控制模型复杂度的效果了。就没必要再使用dropout。


# BatchNorm2d
在4D输入上应用批归一化(NCHW)，论文参考[Batch Normalization: Accelerating Deep Network Training by Reducing Internal Covariate Shift](https://arxiv.org/abs/1502.03167)

**定义**：  
torch.nn.BatchNorm2d(num_features, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True, device=None, dtype=None)

**公式**：  
$$y = \frac{x - \mathrm{E}[x]}{ \sqrt{\mathrm{Var}[x] + \epsilon}} * \gamma + \beta$$

均值和标准偏差在小批量上按维度计算， $γ$ 和 $β$ 是大小为 $C$ 的可学习参数向量(其中 $C$ 为NCHW中的C)。默认情况下， $γ$ 的元素被设置为1， $β$ 的元素被设置为0。标准偏差通过**有偏估计**来计算，相当于torch.var(input, unbiased=False)。

同样在默认情况下，在训练期间，该层保存对其计算出的平均值和方差的估计，然后在评估期间用于归一化。运行估计保持默认动量为0.1。如果track_running_stats设置为False，则该层不会继续运行全局均值和全局无偏方差的估计，而是在评估期间使用批量统计。

**参数**:  
- num_features (int) – $C$ from an expected input of size $(N, C, H, W)$.  $C$ 是输入张量尺寸$(N, C, H, W)$中的 $C$ 。

- eps (float) – a value added to the denominator for numerical stability. Default: 1e-5.  为数值稳定性添加到分母的值。默认值:1e-5

- momentum (float) – the value used for the running_mean and running_var computation. Can be set to None for cumulative moving average (i.e. simple average). Default: 0.1.  用于running_mean（全局均值）和running_var（全局无偏方差）计算的值。累积移动平均，可以设置为None。默认值:0.1。

- affine (bool) – a boolean value that when set to True, this module has learnable affine parameters. Default: True.  一个布尔值，当设置为True时，该模块具有可学习的仿射参数(也就是 $γ$ 和 $β$ )。默认值:True

- track_running_stats (bool) – a boolean value that when set to True, this module tracks the running mean and variance, and when set to False, this module does not track such statistics, and initializes statistics buffers running_mean and running_var as None. When these buffers are None, this module always uses batch statistics in both training and eval modes. Default: True.  当设置为True时，此模块跟踪运行的平均值和方差；当设置为False时，此模块不跟踪此类统计，并将统计缓冲区running_mean和running_var初始化为None。当这些缓冲区为None时，该模块在训练和评估模式中总是使用批量统计。默认值:True

# 图解train模式下的前向传播过程

<p align="center">
<a href="https://raw.githubusercontent.com/itmorn/AI.handbook/main/DL/torch/nn/Normalization/imgs/BatchNorm.svg">
<img src="./imgs/BatchNorm.svg"
    width="2000" /></a></p>


In [137]:
# 手工计算
import torch

input1 = torch.tensor([
    [
        [[1, 6],
         [9, 4]],
        [[12, 18],
         [13, 11]]],
    [
        [[2, 7],
         [3, 8]],
        [[19, 17],
         [15, 11]]
    ]
], dtype=torch.float32)
print("input1:\n", input1, "\n")

# 第1步：按照通道求均值和方差：
VarX, EX = torch.var_mean(input, dim=(0, 2, 3), keepdim=True, unbiased=False)  # NCHW
print("Ex:\n", EX, "\n")
print("VarX:\n", VarX, "\n")

# 第2步：减去均值：
result2 = input1-EX
print("input1-Ex:\n", result2, "\n")

# 第3步：求sqrt(VarX+eps)：
eps = 1e-5
result3 = torch.sqrt(VarX+eps)
print("sqrt(VarX+eps):\n", result3, "\n")

# 第4步：第2步的结果/第3步的结果，完成batch内的数据规范化:
result4 = result2/result3
print("(input1-Ex)/sqrt(VarX+eps):\n", result4, "\n")

# 第5步：使用γ=1，β=0 进行再校正：
γ=1
β=0
result5 = result4 * γ + β
print("[(input1-Ex)/sqrt(VarX+eps)] * γ + β:\n", result5, "\n")


input1:
 tensor([[[[ 1.,  6.],
          [ 9.,  4.]],

         [[12., 18.],
          [13., 11.]]],


        [[[ 2.,  7.],
          [ 3.,  8.]],

         [[19., 17.],
          [15., 11.]]]]) 

Ex:
 tensor([[[[ 5.0000]],

         [[14.5000]]]]) 

VarX:
 tensor([[[[7.5000]],

         [[9.0000]]]]) 

input1-Ex:
 tensor([[[[-4.0000,  1.0000],
          [ 4.0000, -1.0000]],

         [[-2.5000,  3.5000],
          [-1.5000, -3.5000]]],


        [[[-3.0000,  2.0000],
          [-2.0000,  3.0000]],

         [[ 4.5000,  2.5000],
          [ 0.5000, -3.5000]]]]) 

sqrt(VarX+eps):
 tensor([[[[2.7386]],

         [[3.0000]]]]) 

(input1-Ex)/sqrt(VarX+eps):
 tensor([[[[-1.4606,  0.3651],
          [ 1.4606, -0.3651]],

         [[-0.8333,  1.1667],
          [-0.5000, -1.1667]]],


        [[[-1.0954,  0.7303],
          [-0.7303,  1.0954]],

         [[ 1.5000,  0.8333],
          [ 0.1667, -1.1667]]]]) 

[(input1-Ex)/sqrt(VarX+eps)] * γ + β:
 tensor([[[[-1.4606,  0.3651],
          [ 1.

In [149]:
# 调包计算
import torch
import torch.nn as nn

input1 = torch.tensor([
    [
        [[1, 6],
         [9, 4]],
        [[12, 18],
         [13, 11]]],
    [
        [[2, 7],
         [3, 8]],
        [[19, 17],
         [15, 11]]
    ]
], dtype=torch.float32)
print("input1:\n", input1,"\n")

m = nn.BatchNorm2d(num_features=2, eps=1e-5, momentum=1, affine=True, track_running_stats=True)
m.train()

print("nn.BatchNorm2d默认初始化可学习参数γ=1:\n", m.weight,"\n")
print("nn.BatchNorm2d默认初始化可学习参数β=0:\n", m.bias,"\n")

output = m(input1)
print("output:\n", output,"\n") # 结果和手工计算一致

input1:
 tensor([[[[ 1.,  6.],
          [ 9.,  4.]],

         [[12., 18.],
          [13., 11.]]],


        [[[ 2.,  7.],
          [ 3.,  8.]],

         [[19., 17.],
          [15., 11.]]]]) 

nn.BatchNorm2d默认初始化可学习参数γ=1:
 Parameter containing:
tensor([1., 1.], requires_grad=True) 

nn.BatchNorm2d默认初始化可学习参数β=0:
 Parameter containing:
tensor([0., 0.], requires_grad=True) 

output:
 tensor([[[[-1.4606,  0.3651],
          [ 1.4606, -0.3651]],

         [[-0.8333,  1.1667],
          [-0.5000, -1.1667]]],


        [[[-1.0954,  0.7303],
          [-0.7303,  1.0954]],

         [[ 1.5000,  0.8333],
          [ 0.1667, -1.1667]]]], grad_fn=<NativeBatchNormBackward0>) 



# 图解train模式下维护全局均值方差
我们知道，在模型的train过程中，每次前向传播会处理一个batch的样本，此时，我们可以在这个小批次上统计每个通道的均值和方差，进而完成小批次内的规范化，即：
$$y = \frac{x - \mathrm{E}[x]}{ \sqrt{\mathrm{Var}[x] + \epsilon}} $$

然而，在推理的过程中，我们往往都是对一个样本进行推理，这时候就没办法进行规范化了。那有什么解决方案吗？当然。比如我们可以计算全部训练数据的均值和方差，供推理的时候使用。这个方法可行，且比较准确的反馈了数据整体的均值和无偏方差，但问题是计算成本比较大。torch中采用的是**移动平均**的计算方法。下面简述一下步骤：  

1. 初始化：全局均值=0，全局方差=1。
2. 计算：第 i 批数据计算出来的均值=EX_i，方差=VarX_i.
3. 更新：全局均值 = EX_i * momentum + 全局均值 * (1-momentum).
4. 更新：全局方差 = VarX_i * momentum + 全局方差 * (1-momentum).
5. 重复2,3,4直到训练结束，便可得到**全局均值**和**全局方差**。

<p align="center">
<a href="https://raw.githubusercontent.com/itmorn/AI.handbook/main/DL/torch/nn/Normalization/imgs/BatchNorm2.svg">
<img src="./imgs/BatchNorm2.svg"
    width="2000" /></a></p>


In [159]:
# 调包计算训练过程中BN维护的全局均值和方差
import torch
import torch.nn as nn

input1 = torch.tensor([
    [
        [[1, 6],
         [9, 4]],
        [[12, 18],
         [13, 11]]],
    [
        [[2, 7],
         [3, 8]],
        [[19, 17],
         [15, 11]]
    ]
], dtype=torch.float32)

m = nn.BatchNorm2d(num_features=2, eps=1e-5, momentum=0.1, affine=True, track_running_stats=True)
m.train()

print("nn.BatchNorm2d默认初始化全局均值:\n", m.running_mean,"\n")
print("nn.BatchNorm2d默认初始化全局方差:\n", m.running_var,"\n\n\n")

output = m(input1)

print("nn.BatchNorm2d全局均值 after 1 batch:\n", m.running_mean,"\n")
print("nn.BatchNorm2d全局方差 after 1 batch:\n", m.running_var,"\n\n\n")

output = m(input1)

print("nn.BatchNorm2d全局均值 after 2 batch:\n", m.running_mean,"\n")
print("nn.BatchNorm2d全局方差 after 2 batch:\n", m.running_var,"\n\n\n") # 和上图中的结果一致

nn.BatchNorm2d默认初始化全局均值:
 tensor([0., 0.]) 

nn.BatchNorm2d默认初始化全局方差:
 tensor([1., 1.]) 



nn.BatchNorm2d全局均值 after 1 batch:
 tensor([0.5000, 1.4500]) 

nn.BatchNorm2d全局方差 after 1 batch:
 tensor([1.7571, 1.9286]) 



nn.BatchNorm2d全局均值 after 2 batch:
 tensor([0.9500, 2.7550]) 

nn.BatchNorm2d全局方差 after 2 batch:
 tensor([2.4386, 2.7643]) 





# eval模式下的前向传播
推理过程会使用训练模式下维护的全局均值和全局无偏方差，且不会再改变它们。

In [182]:
# 调包计算训练过程中BN维护的全局均值和方差
import torch
import torch.nn as nn

input1 = torch.tensor([
    [
        [[1, 6],
         [9, 4]],
        [[12, 18],
         [13, 11]]],
    [
        [[2, 7],
         [3, 8]],
        [[19, 17],
         [15, 11]]
    ]
], dtype=torch.float32)

m = nn.BatchNorm2d(num_features=2, eps=1e-5, momentum=0.1, affine=True, track_running_stats=True)

m.train()
output = m(input1)
output = m(input1)

print("===================train模式下训练2个batch后的全局均值和全局无偏方差=============")
print("nn.BatchNorm2d全局均值 after 2 batch:\n", m.running_mean,"\n")
print("nn.BatchNorm2d全局方差 after 2 batch:\n", m.running_var)
print("==============================================================================\n\n\n")


m.eval()
output = m(input1)
output = m(input1)
print("===================eval模式下推理2个batch后的全局均值和全局无偏方差=============")
print("nn.BatchNorm2d全局均值 after 2 batch:\n", m.running_mean,"\n")
print("nn.BatchNorm2d全局方差 after 2 batch:\n", m.running_var)
print("==============================================================================\n\n\n")
# 可以看到eval模式下全局均值和全局无偏方差不再改变。
print("output:\n", output,"\n") 

# 手工计算
result2 = input1-m.running_mean
eps = 1e-5
result3 = torch.sqrt(m.running_var+eps)
result4 = result2/result3
γ=1
β=0
result5 = result4 * γ + β
print("result5:\n", result5,"\n")  #可以看到调包计算和手工计算的结果一致


===================train模式下训练2个batch后的全局均值和全局无偏方差=============
nn.BatchNorm2d全局均值 after 2 batch:
 tensor([0.9500, 2.7550]) 

nn.BatchNorm2d全局方差 after 2 batch:
 tensor([2.4386, 2.7643])



===================eval模式下推理2个batch后的全局均值和全局无偏方差=============
nn.BatchNorm2d全局均值 after 2 batch:
 tensor([0.9500, 2.7550]) 

nn.BatchNorm2d全局方差 after 2 batch:
 tensor([2.4386, 2.7643])



output:
 tensor([[[[0.0320, 3.2339],
          [5.1550, 1.9531]],

         [[5.5605, 9.1693],
          [6.1620, 4.9590]]],


        [[[0.6724, 3.8742],
          [1.3128, 4.5146]],

         [[9.7707, 8.5678],
          [7.3649, 4.9590]]]], grad_fn=<NativeBatchNormBackward0>) 

result5:
 tensor([[[[ 0.0320,  1.9517],
          [ 5.1550,  0.7488]],

         [[ 7.0761,  9.1693],
          [ 7.7165,  4.9590]]],


        [[[ 0.6724,  2.5532],
          [ 1.3128,  3.1547]],

         [[11.5587,  8.5678],
          [ 8.9972,  4.9590]]]]) 



# 参考资料
[图解深度学习与神经网络：从张量到TensorFlow实现》_张平](https://item.jd.com/12429187.html)
[28 批量归一化【动手学深度学习v2】](https://www.bilibili.com/video/BV1X44y1r77r)